In [1]:
import numpy as np
import pandas as pd

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_2016 = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/2016.csv')
df_2017 = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/2016.csv')
df_2018 = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/2016.csv')

In [5]:
localityNo = 11652 # Kjeppevikholmen

In [6]:
subset = df.loc[df['localityNo'] == localityNo]

print(subset)

           id  localityNo  year  week  hasReportedLice  hasMechanicalRemoval  \
4955   174427       11652  2016    11             True                 False   
5414   174428       11652  2016    12             True                 False   
5862   174429       11652  2016    13             True                 False   
6325   174430       11652  2016    14             True                 False   
6789   174431       11652  2016    15             True                 False   
7269   174432       11652  2016    16             True                 False   
7754   174433       11652  2016    17             True                 False   
8251   174434       11652  2016    18             True                 False   
8753   174435       11652  2016    19             True                 False   
9260   174436       11652  2016    20             True                 False   
9769   174437       11652  2016    21             True                 False   
10279  174438       11652  2016    22   